<a href="https://colab.research.google.com/github/jasonwong-lab/HKU-Practical-Bioinformatics/blob/main/Variant_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HKU Practical Bioinformatics 2025 - Variant Calling

*by Prof. Jason Wong*

This lecture aims to demonstrate how variants/mutations are called from aligned next-generation sequencing data.

#Set working directory

By default, the working directory will be My Drive/PB_course.

In [ ]:
# Set working pathway to your own google drive doc (~ 1 min)
from google.colab import drive
drive.mount('/content/gdrive')                         # if using for the first time, you be requested to grant permission to link your Google Drive

import os
try:
  os.mkdir("/content/gdrive/My Drive/PB_course/")         # change this path if necessary
except FileExistsError:
  print("directory already exist. OK to continue")
os.chdir("/content/gdrive/My Drive/PB_course/")

In [ ]:
!pwd

#Package installation and downloads for workshop (~ 5 minutes)

**IMPORTANT: Every time you connect to Google Colab, you have to perform these set up steps again**

In [ ]:
# Install conda (~ 1 min). There will be a message saying that the session has crashed but don't worry about this. This is due to the session restarting following conda installation
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# Install lofreq (~ 1 min)
!conda install -c bioconda lofreq

In [ ]:
# Install GATK in a new environment (to specify the exapct openjdk and python version) ~1.5 mins
!conda create -y -n gatkenv python=3.10
!conda config --env --set channel_priority strict
!conda install -y -n gatkenv -c conda-forge -c bioconda gatk4=4.6.2.0 openjdk=17

In [ ]:
# Check that gatk is installed properly
!conda run -n gatkenv gatk --version

In [ ]:
!conda install -y -c conda-forge -c bioconda \
  samtools=1.20 bcftools=1.20 htslib=1.20 bedtools=2.31.1 bamtools

In [ ]:
# Check installation ran correctly
!samtools --version | head -1

In [ ]:
# Install igv-notebook
import sys
print(sys.version, sys.executable)
!{sys.executable} -m pip install -U igv-notebook
import igv_notebook

In [ ]:
# Check if you already have the BAM files generated from last lecture + exercise
!ls -l /content/gdrive/MyDrive/PB_course/Datasets/BAM

In [ ]:
# If you don't have WXS_example_*_sorted.bam, run this cell
import os
os.chdir("/content/gdrive/My Drive/PB_course/Datasets/")
if os.path.isfile("/content/gdrive/MyDrive/PB_course/Datasets/BAM/WXS_example_sorted.bam"):    # check if the file exist
  print("WXS_example_*_sorted.bam files already exist, OK to continue")
else:
  !pip install gdown
  !gdown -O WXS_BAMs.zip https://drive.google.com/uc?id=1Asekyf-W-XAIjrCi4ok-xEIdy625i943
  !unzip WXS_BAMs.zip
  !rm WXS_BAMs.zip

In [ ]:
# Download reference sequence
# Double check that we are in the right directory (~ 30s)
import os
os.chdir("/content/gdrive/MyDrive/PB_course")                     # change this path if necessary

import os
if os.path.isfile("/content/gdrive/MyDrive/PB_course/DB_trunc/chr2.fa"):    # check if the file exist
  print("reference file already exist, OK to continue.")
else:
  !pip install gdown
  !gdown -O DB_trunc.zip https://drive.google.com/uc?id=1aRJVznjy5WLQ5Dc0DT9c6NiXw64HdoKr # download if file not exist
  # unzip fasta file
  !unzip -o DB_trunc.zip
  # remove the zip file after extraction
  !rm DB_trunc.zip

!ls -l ./DB_trunc/

In [ ]:
# Download reference sequence for GATK
import os
os.chdir("/content/gdrive/MyDrive/PB_course/Datasets")                     # change this path if necessary

import os
if os.path.isfile("/content/gdrive/MyDrive/PB_course/Datasets/GATK_reference"):    # check if the file exist
  print("reference file already exist, OK to continue.")
else:
  !pip install gdown
  !gdown -O GATK_reference.zip https://drive.google.com/uc?id=1Wxkng_IlPuw_EOgDo80LyEE4fT37ZV4W # download if file not exist
  # unzip fasta file
  !unzip -n GATK_reference.zip
  # remove the zip file after extraction
  !rm GATK_reference.zip

!ls -l ./GATK_reference/


#Variant Calling

## Variant calling command line: Lofreq

In [ ]:
!lofreq

In [ ]:
# Variant calling using LoFreq
!lofreq call

In [ ]:
# Step 1 – Create directory to store VCF files. Make sure we are in the Datasets directory.
%cd /content/gdrive/My Drive/PB_course/Datasets/
!mkdir VCF/

In [ ]:
# Step 2 – Run LoFreq call
!rm VCF/*.vcf # removing existing VCF files in case lofreq call was already ran previously
!lofreq call ./BAM/WXS_example_sorted.bam -o ./VCF/WXS_germline_nofilter.vcf -f ../DB_trunc/chr2.fa --verbose --no-default-filter

In [ ]:
# Step 3 - Run LoFreq filter
!lofreq filter --print-all -i ./VCF/WXS_germline_nofilter.vcf -o ./VCF/WXS_germline_filter.vcf

In [ ]:
# Check result
!head -n 30 ./VCF/WXS_germline_filter.vcf

In [ ]:
import igv_notebook

igv_notebook.init()

b = igv_notebook.Browser(
    {
        "genome": "hg38",
        "locus": "chr2:5876477-5876517"
    }
)

b.load_track(
    {
        "name": "WXS",
        "path": "./BAM/WXS_example_sorted.bam",
        "indexPath": "./BAM/WXS_example_sorted.bam.bai",
        "format": "bam",
        "type": "alignment"
    })

## Variant calling command line: Mutect2
1. Alignment post-processing
2. Run alignment metrics generation
3. Variant Calling
4. Normalisation and Filtering

### Alignment post-processing

In [ ]:
# Step 1 – Create directory to store GATK output files. Make sure we are in the Datasets directory.
%cd /content/gdrive/My Drive/PB_course/Datasets/
!mkdir GATK/

In [ ]:
# Step 2 – Add read groups (RG), tags for downstream tools to run properly
!conda run -n gatkenv gatk AddOrReplaceReadGroups \
  -I ./BAM/WXS_example_sorted.bam \
  -O ./GATK/WXS_example_sorted.RG.bam \
  --RGID RG1 --RGLB lib1 --RGPL ILLUMINA --RGPU unit1 \
  --RGSM WXS_example --CREATE_INDEX true

In [ ]:
# View our output
!samtools view -H ./GATK/WXS_example_sorted.RG.bam | grep '^@RG'

In [ ]:
# Step 3 - Mark PCR/optical duplicates so they don't inflate evidence
!conda run -n gatkenv gatk MarkDuplicates\
  -I ./GATK/WXS_example_sorted.RG.bam \
  -O ./GATK/WXS_example_sorted.MD.bam \
  -M  ./GATK/WXS_example_sorted.MD.stat \
  --CREATE_INDEX true

In [ ]:
# Create dictionary for the reference genome file (metadata file)
if os.path.isfile("/content/gdrive/MyDrive/PB_course/DB_trunc/chr2.dict"):    # check if the file exist
  print("dictionary file already exist, OK to continue.")
else:
  !conda run -n gatkenv gatk CreateSequenceDictionary \
    -R ../DB_trunc/chr2.fa

In [ ]:
# Step 4 - Base Quality Score Recalibration: statistically corrects systematic base-quality errors using known sites
# Step 4a. Build Recal model
!conda run -n gatkenv gatk BaseRecalibrator \
  -R ../DB_trunc/chr2.fa -I ./GATK/WXS_example_sorted.MD.bam \
  -L ./GATK_reference/intervals.bed -ip 10 \
  --known-sites ./GATK_reference/dbsnp138.hg38.subset.vcf \
  --known-sites ./GATK_reference/mills.hg38.subset.vcf.gz \
  -O ./GATK/WXS_example_sorted_recal_data.table

In [ ]:
# Step 4b. Apply recalibration to bam file
!conda run -n gatkenv gatk ApplyBQSR \
  -R ../DB_trunc/chr2.fa -I ./GATK/WXS_example_sorted.MD.bam \
  --bqsr-recal-file ./GATK/WXS_example_sorted_recal_data.table \
  -O ./GATK/WXS_example_sorted.MD_BR.bam


In [ ]:
import igv_notebook

igv_notebook.init()

b = igv_notebook.Browser(
    {
        "genome": "hg38",
        "locus": "chr2:5876477-5876517"
    }
)

b.load_track(
    {
        "name": "WXS",
        "path": "./BAM/WXS_example_sorted.bam",
        "indexPath": "./BAM/WXS_example_sorted.bam.bai",
        "format": "bam",
        "type": "alignment"
    })
b.load_track(
    {
        "name": "WXS MD BR",
        "path": "./GATK/WXS_example_sorted.MD_BR.bam",
        "indexPath": "./GATK/WXS_example_sorted.MD_BR.bai",
        "format": "bam",
        "type": "alignment"
    })

###Run Alignment Metrics generation

In [ ]:
!mkdir ./GATK/QC

In [ ]:
# QC1. Insert-size & basic BAM stats
!bamtools stats -insert -in ./GATK/WXS_example_sorted.MD_BR.bam > ./GATK/QC/WXS_example_sorted.QC.bamtools_stat.txt


In [ ]:
# QC2. Picard multi-metrics (alignment/quality/insert-size PDFs)
!conda run -n gatkenv gatk CollectReadCounts -I ./GATK/WXS_example_sorted.MD_BR.bam -L ./GATK_reference/intervals.bed --interval-merging-rule OVERLAPPING_ONLY --format TSV -O ./GATK/QC/WXS_example_sorted.QC.read_counts.tsv

In [ ]:
# QC3. Per-interval read counts (for coverage)
!conda run -n gatkenv gatk CollectMultipleMetrics -I ./GATK/WXS_example_sorted.MD_BR.bam -O ./GATK/QC/WXS_example_sorted.QC.multiple_metrics

In [ ]:
# QC4. Hybrid-capture metrics (bait/target coverage)
!conda run -n gatkenv gatk CollectHsMetrics -I ./GATK/WXS_example_sorted.MD_BR.bam -O ./GATK/QC/WXS_example_sorted.QC.hs_metrics.txt -R ../DB_trunc/chr2.fa -BI ./GATK_reference/intervals.interval_list -TI ./GATK_reference/intervals.interval_list


### Variant calling: Mutect2

In [ ]:
# Call variants using Mutect2
!conda run -n gatkenv gatk Mutect2 \
  -R ../DB_trunc/chr2.fa \
  -I ./GATK/WXS_example_sorted.MD_BR.bam \
  -tumor WXS_example \
  -L ./GATK_reference/intervals.bed \
  -ip 10 --min-base-quality-score 20 \
  --germline-resource ./GATK_reference/af-only-gnomad.hg38.subset.vcf.gz \
  --f1r2-tar-gz ./GATK/WXS_example_sorted.Mutect2.f1r2.tar.gz  \
  -O ./GATK/WXS_example_sorted.Mutect2.unfiltered.vcf


In [ ]:
# Learn strand/orientation artifact model
!conda run -n gatkenv gatk LearnReadOrientationModel -I ./GATK/WXS_example_sorted.Mutect2.f1r2.tar.gz -O ./GATK/WXS_example_sorted.Mutect2.read-orientation-model.tar.gz
# Estimate contamination from common SNPs
!conda run -n gatkenv gatk GetPileupSummaries -I ./GATK/WXS_example_sorted.MD_BR.bam -V ./GATK_reference/small_exac_common_3.hg38.subset.vcf.gz -L ./GATK_reference/small_exac_common_3.hg38.subset.vcf.gz -O ./GATK/WXS_example_sorted.Mutect2.getpileupsummaries.table
!conda run -n gatkenv gatk CalculateContamination -I ./GATK/WXS_example_sorted.Mutect2.getpileupsummaries.table -O ./GATK/WXS_example_sorted.Mutect2.calculatecontamination.table


In [ ]:
# Probabilistic filtering of calls
!conda run -n gatkenv gatk FilterMutectCalls \
  -R ../DB_trunc/chr2.fa \
  -V ./GATK/WXS_example_sorted.Mutect2.unfiltered.vcf \
  --contamination-table ./GATK/WXS_example_sorted.Mutect2.calculatecontamination.table \
  --ob-priors ./GATK/WXS_example_sorted.Mutect2.read-orientation-model.tar.gz \
  -O ./GATK/WXS_example_sorted.Mutect2.filter1.vcf


### Normalisation and Filtering

In [ ]:
# Post-calling normalization and indexing
!bcftools norm -m-both ./GATK/WXS_example_sorted.Mutect2.filter1.vcf | bcftools norm -f ../DB_trunc/chr2.fa | bgzip -c > ./GATK/WXS_example_sorted.Mutect2.filter1.norm.vcf.gz
!tabix -p vcf ./GATK/WXS_example_sorted.Mutect2.filter1.norm.vcf.gz

In [ ]:
# Applying filters on depth, VAF, rarity
!bcftools filter -i 'AF>=0.02 && FORMAT/DP>=50 && (INFO/POPAF>=2 || INFO/POPAF=".")'\
  ./GATK/WXS_example_sorted.Mutect2.filter1.norm.vcf.gz \
| bgzip -c > ./GATK/WXS_example_sorted.Mutect2.final.vcf.gz
!tabix -p vcf ./GATK/WXS_example_sorted.Mutect2.final.vcf.gz

In [ ]:
# Cleaning up vcf, extracing allele depth info only
!bcftools annotate -x INFO,^FORMAT/AD ./GATK/WXS_example_sorted.Mutect2.final.vcf.gz -O z -o ./GATK/WXS_example_sorted.Mutect2.final.AD.vcf.gz
!tabix -p vcf ./GATK/WXS_example_sorted.Mutect2.final.AD.vcf.gz

In [ ]:
# View our results
!bcftools view -H ./GATK/WXS_example_sorted.Mutect2.final.AD.vcf.gz | wc -l
!bcftools view -H ./GATK/WXS_example_sorted.Mutect2.final.AD.vcf.gz | head

In [ ]:
import igv_notebook

igv_notebook.init()

b = igv_notebook.Browser(
    {
        "genome": "hg38",
        "locus": "chr2:47,782,145-47,807,953"
    }
)

b.load_track(
    {
        "name": "WXS",
        "path": "./GATK/WXS_example_sorted.MD_BR.bam",
        "indexPath": "./GATK/WXS_example_sorted.MD_BR.bai",
        "format": "bam",
        "type": "alignment"
    })

b.load_track(
    {
        "name": "VCF",
        "path": "./GATK/WXS_example_sorted.Mutect2.filter1.norm.vcf.gz",
        "indexPath": "./GATK/WXS_example_sorted.Mutect2.filter1.norm.vcf.gz.tbi",
        "format": "vcf"
    })